# 20210119
- review image 데이터를 정답지(Y, N)으로 분류하여 별도 디렉토리에 저장

In [ ]:
# 주피터노트북 화면 넓게 보기 위한 용도
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import boto3
import s3fs

import pathlib

import pymysql
from PIL import Image

from dotenv import load_dotenv
import os
load_dotenv() # .env 파일에서 정보를 갖고 오기 위함

In [ ]:
!pip install python-dotenv
# https://medium.com/@namseok.yoo/django-%EC%97%90%EC%84%9C-python-dotenv-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-6d9b58a939ec

In [ ]:
conn = pymysql.connect(database=os.getenv('database'), user=os.getenv('user'), password=os.getenv('password'), host= 'localhost', port=3306)
qry = '''
    SELECT a.goods_no, a.est_no, b.image, a.point_yn
    FROM 
    (
        SELECT goods_no, no as est_no, point_yn
        FROM goods_estimate
        WHERE date_format(regi_date, '%Y%m%d') between '20201202' and '20201204'
        AND type = 'style' 
        AND point_yn = 'N'
    ) a
    JOIN
    (
        SELECT est_no, image
        FROM goods_estimate_image
        WHERE date_format(rt, '%Y%m%d') between '20201202' and '20201204'
        GROUP BY est_no, image HAVING count(img_no) = 1
    ) b
    ON a.est_no = b.est_no
'''
df = pd.read_sql_query(qry, conn)

In [ ]:
df['point_yn'].describe()

In [ ]:
df.head()

# 20210120
### S3에서 데이터 fetch하기 
- tf.keras.preprocessing.image_dataset_from_directory를 고려했으나 s3에서 바로 동작하지 않아 -> 아래 아티클 참고하여 진행
- https://medium.com/analytics-vidhya/custom-keras-generator-fetching-images-from-s3-to-train-neural-network-4e98694de8ee

In [ ]:
## DB에 저장된 image path를 사용하면 되므로, 다음 과정은 불필요
# import pandas as pd
# import boto3
# from time import time

# bucket_name = 'bucket'
# client = boto3.client('s3')

# test = client.get_paginator('list_objects_v2') # prefix에 특정 key 가져오는거라, 이게 더 복잡한듯
# page_iterator = test.paginate(Bucket=os.getenv('bucket'), Prefix=os.getenv('estimate_prefix')
# g = pd.Series()
# a = time()
# n = 0
# for page in page_iterator:
#     m = [d['Key'] for d in page['Contents'] if d['Key'][-4:] == '.jpg'] 
#     print(m)
#     if len(m) > 0:
#         g = g.append(pd.Series(m))
#     if len(g) == 10000:
#         break

# g.reset_index(drop=True, inplace=True)

In [ ]:
# from keras.preprocessing.image import load_img, img_to_array
# import io

# def fetch_input(path, s3):
#     object = s3.Object(bucket_name, path)
#     img = load_img(io.BytesIO(object.get()['Body'].read()))
#     return(img)


# def preprocess_input(img):
#     image = img.resize((128,128))
#     array = img_to_array(image)
#     return(array)

# # I’m using the keras loading module, but properly the common PIL package can be used as well.

In [ ]:
# def s3_image_generator(files, batch_size = 16):
#     s3 = boto3.resource('s3')
#     while True:
#         batch_paths = np.array(files)
#         batch_input = []
#         batch_output = [0] * len(files)
#         for input_path in batch_paths:
#             input = fectch_input(input_path, s3)
#             input = preprocess_input(input)
#             batch_input += [ input ]
#         batch_x = np.array( batch_input )
#         batch_y = np.array( batch_output )
#         yield( batch_x, batch_y )

In [ ]:
# first = 1
# a = time()
# s3 = boto3.resource('s3')
# for path in g[0:1000]:
#     x_batch=[]
#     object = s3.Object(bucket_name,path)
#     img = load_img(io.BytesIO(object.get()['Body'].read()))
#     image = img.resize((128,128))
#     array = img_to_arrayㅇ(image)
#     preds=(model.predict_on_batch(array))
#     if first==1:
#         predsA=preds.copy()
#         first=0
#     else:
#         predsA=np.append(predsA,preds,axis=0)
# time()-a

s3 -> local -> fetch 밖에 답이 없는듯

- 데이터 디렉토리 아래 클래스별로 이미지 나눠서 저장

In [ ]:
# 아래 코드만 정상동작

def get_image_from_s3(img_path, size=128):
    file = boto3.resource('s3').Object(os.getenv('bucket'), img_path[1:]).get()
    img = Image.open(file['Body']).resize((size, size))
    return img

def save_image(img, path, is_array=False):
    if is_array:
        Image.fromarray(img).save(path)
    else:
        img.save(path)

    return True

def proc_image(img_path, est_no, point_yn):
    try:
        img = get_image_from_s3(img_path)
        local_img_path = f'./data/{point_yn.upper()}/{est_no}.jpg'
        _ = save_image(img, local_img_path)
        img_array = np.array(img) # convert image to numpy array
        return img_array
    except:
        print('error: ', img_path)

In [ ]:
df.head()

In [ ]:
images_array = df.tail(2000).head(1000).apply(lambda x: proc_image(x['image'], x['est_no'], x['point_yn']), axis=1) # 해당 path에 없는 이미지가 있나봄 - 마지막에 에러남 

# 20210121

In [ ]:
data_dir = pathlib.Path(path) # 하위 디렉토리 하나당 하나의 클래스
data_dir

In [ ]:
img_size = 128
batch = 32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, 
    subset='training',
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, 
    subset='validation',
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

num_classes = len(class_names)
num_classes

In [ ]:
import matplotlib.pyplot as plt
for images, labels in train_ds.take(1):
    print(images.shape)  # train_dataset - 한 배치당 32개 데이터 포함 (batch_size=32)
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE) 
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [ ]:
IMG_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

In [ ]:
feature_batch = base_model(image_batch)  # base_model의 output: (None, 5, 5, 1280)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()  # avg pooling 전략
feature_batch_average = global_average_layer(feature_batch)

prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = normalization_layer(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
initial_epochs = 10
loss0, accuracy0 = model.evaluate(val_ds)

history = model.fit(train_ds, epochs=initial_epochs, validation_data=val_ds)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0, 1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
from PIL import Image

# Y인데 못 맞춘 이미지
class_name = 'Y'
for i in y_jpgs:
    img = keras.preprocessing.image.load_img(
        pathlib.Path(f'{path}{class_name}/{i}'), target_size=(128, 128)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    if model.predict(img_array)[0][0] < 0.2:
        img_path = f'{path}{class_name}/{i}'
        print(img_path)
        display(Image.open(f'{path}{class_name}/{i}'))
#     print(model.predict(img_array)[0][0])  ## 이래서 sigmoid를 해야하는거구나

In [ ]:
# N인데 못 맞춘 이미지
class_name = 'N'
for i in n_jpgs:
    img = keras.preprocessing.image.load_img(
        pathlib.Path(f'{path}{class_name}/{i}'), target_size=(128, 128)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    if model.predict(img_array)[0][0] < 0.3:
        img_path = f'{path}{class_name}/{i}'
        print(img_path)
        display(Image.open(f'{path}{class_name}/{i}'))
#     print(model.predict(img_array)[0][0])  ## 이래서 sigmoid를 해야하는거구나

In [ ]:
df[df['est_no']==13545123] # https://store.musinsa.com/app/reviews/views/13545123